## Detection algorithm

Now, we take the simulated signal and use it do detect wether changes have ocurred.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import yaml
import numpy as np
import pandas as pd

from src import paths
from src.inference.pixel_simulation import PixelSimulation
from src.inference.pandas_wrapper import PandasWrapper

## Single pixel change detection

In [54]:
with open(paths.config_dir("params.yaml"), "r") as file:
    params = yaml.safe_load(file)

change_detection_forecasted_steps = params["change_detection_forecasted_steps"]
N_values = params["N_values"]
k_values = params["k_values"]
offset_values = params["offset_values"]

In [77]:
with open(paths.results_simulations_dir("pixel_simulations.pk"), "rb") as file:
    results_simulations = pickle.load(file)

simulations_df = PandasWrapper(results_simulations, object_name="simulation")

In [78]:
simulations_df

                  simulation
ID  IDPix                   
31  957    LazyLoaded: False
59  1496   LazyLoaded: False
28  773    LazyLoaded: False
245 826    LazyLoaded: False
300 5084   LazyLoaded: False
241 239    LazyLoaded: False
373 1441   LazyLoaded: False
392 3906   LazyLoaded: False

In [50]:
pixel_simulation = simulations_df.iloc[0]

In [7]:
N = 10
k = 0.8
offset = 1

detections = []
dates = []

num_of_detections = (len(pixel_simulation) - 1) - offset
num_of_windows = change_detection_forecasted_steps - offset

for i in range(num_of_detections):
    for j in range(num_of_windows):
        detection_window = pixel_simulation[i + offset].iloc[j:j+N].to_numpy(
        ) < k * pixel_simulation[i].iloc[j: j + N].to_numpy()

        flag = np.all(detection_window)
        date = pixel_simulation[i].index[0]

        detections.append(float(flag))
        dates.append(date)

detection_series = pd.Series(detections, index=pd.to_datetime(dates))

In [51]:
detection_series

2016-07-24    0.0
2016-07-24    0.0
2016-07-24    0.0
2016-07-24    0.0
2016-07-24    0.0
             ... 
2021-12-12    0.0
2021-12-12    0.0
2021-12-12    0.0
2021-12-12    0.0
2021-12-12    0.0
Length: 14382, dtype: float64

In [9]:
import numpy as np
import pandas as pd
from typing import List, Tuple


class PixelChangeDetection:

    def __init__(self, index: Tuple[int, int],
                 detected_changes: pd.Series,
                 offset: int,
                 N: int,
                 k: float):

        self.index = index
        self.offset = offset
        self.N = N
        self.k = k
        self.detected_changes = detected_changes

    def __repr__(self) -> str:

        return f"PixelChangeDetection_{self.index[0]}_{self.index[1]}"

    def __str__(self) -> str:

        return f"PixelChangeDetection_{self.index[0]}_{self.index[1]}"


def detect_changes_in_pixel(
    pixel_simulation: PixelSimulation,
    N: int,
    k: float,
    offset: int,
    forecasted_steps: int,
) -> PixelChangeDetection:
    """
    Detects changes in a pixel simulation.

    Parameters
    ----------
    pixel_simulation : PixelSimulation
        The PixelSimulation object containing the actual and predicted NDVI signals.
    N : int
        The length of the detection window.
    k : float
        The threshold factor for detecting significant changes.
    offset : int
        The offset between the baseline signal and the predicted signal to compare.
    forecasted_steps : int
        The number of steps in each forecast.

    Returns
    -------
    pd.Series
        A Series of binary values (0 or 1) indicating significant changes, indexed by timestamps.
    """
    detections = []
    dates = []

    num_of_detections = (len(pixel_simulation) - 1) - offset
    num_of_windows = forecasted_steps - offset

    for i in range(num_of_detections):
        for j in range(num_of_windows):
            detection_window = pixel_simulation[i + offset].iloc[j:j+N].to_numpy(
            ) < k * pixel_simulation[i].iloc[j:j+N].to_numpy()

            flag = np.all(detection_window)
            date = pixel_simulation[i].index[j]

            detections.append(float(flag))
            dates.append(date)

    detection_series = pd.Series(detections, index=pd.to_datetime(dates))

    pixel_change_detection = PixelChangeDetection(
        index=pixel_simulation.index,
        offset=offset,
        N=N,
        k=k,
        detected_changes=detection_series,
    )

    return pixel_change_detection

In [10]:
detections = detect_changes_in_pixel(
    pixel_simulation, N=10, k=0.8, offset=1, forecasted_steps=52)

In [11]:
detections

PixelChangeDetection_31_957

## Multiple pixels change detection

We now want a paralelized routine for detecting changes over parameters combinations

In [14]:
from itertools import product
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm


def detect_changes_in_pixel_caller(args):
    pixel_simulation, N, k, offset, forecasted_steps = args
    return detect_changes_in_pixel(pixel_simulation, N, k, offset, forecasted_steps)


def parallel_detect_changes_in_pixels(simulations: PandasWrapper, num_processes: int) -> List[PixelChangeDetection]:

    pixel_indices = simulations.index

    args_list = list(product(pixel_indices, N_values, k_values,
                     offset_values, [change_detection_forecasted_steps]))
    args_list = [(simulations.loc[id], n, k, offset, steps)
                 for id, n, k, offset, steps in args_list]

    results = []
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(
            detect_changes_in_pixel_caller, args) for args in args_list]

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            results.append(future.result())

    return results

In [15]:
results = parallel_detect_changes_in_pixels(simulations_df, num_processes=8)

Processing: 100%|██████████| 200/200 [01:20<00:00,  2.47it/s]


In [76]:
results_by_params = {}
df_results_by_params = {}

for result in results:
    params_index = (result.N, result.k)
    pixel_index = result.index
    
    if params_index not in results_by_params:
        results_by_params[params_index] = {}
        
    results_by_params[params_index][pixel_index] = result


for params_index in results_by_params:
    results_dict = results_by_params[params_index]
    df_results_by_params[params_index] = PandasWrapper(results_dict, "detection")

# Save results by params
